# Extraindo dados do IBGE com sidrapy, e criando dashboards automáticos.

---

### 1 - Importação das bibliotecas.

In [1]:
import pandas as pd
import sidrapy

### 2 - Extração a base de dados.

In [2]:
data = sidrapy.get_table(table_code='1086', 
                         territorial_level='3',
                         ibge_territorial_code='all',
                         variable='283',
                         period='all')

### 3 - Manipulação da base bruta.

Visualização inicial da BASE.

In [3]:
data.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N,D5C,D5N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Unidade da Federação (Código),Unidade da Federação,Trimestre (Código),Trimestre,Variável (Código),Variável,Tipo de inspeção (Código),Tipo de inspeção,Referência temporal (Código),Referência temporal
1,3,Unidade da Federação,25,Mil litros,54392,11,Rondônia,199701,1º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
2,3,Unidade da Federação,25,Mil litros,54822,11,Rondônia,199702,2º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
3,3,Unidade da Federação,25,Mil litros,68027,11,Rondônia,199703,3º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
4,3,Unidade da Federação,25,Mil litros,84671,11,Rondônia,199704,4º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre


Definição do cabeçalho como colula.

In [4]:
data.columns = data.iloc[0]

Fatiando a base e selecionando as colunas necessárias.

In [5]:
df = data.iloc[1:,[7,6,4]].copy()

Criação da coluna 'DATA', a partir da coluna 'Trimestre (Código)'

In [6]:
df['DATA'] = pd.to_datetime(df['Trimestre (Código)'].str.slice(0,4) + '-' + df['Trimestre (Código)'].str.slice(4,6))

Dropando a coluna 'Trimestre (Código)', pois a coluna 'DATA' já foi construída.

In [7]:
df = df.drop(columns='Trimestre (Código)')

Renomeação das colunas, e alteração das posições.

In [8]:
df = df.rename(columns={'Unidade da Federação':'UF', 'Valor':'VALOR'})[['DATA','UF','VALOR']]

Transformando a coluna de 'VALOR' em inteiro.

In [10]:
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce').fillna(0).astype(int)

In [11]:
df['UF'] = df['UF'].astype('string')

### 4 - Conferindo os dtypes e se existe valores nulos/ausentes.

Verificando se há valores nulos/ausentes nas colunas.

In [12]:
df.isnull().sum()

0
DATA     0
UF       0
VALOR    0
dtype: int64

Conferindo os tipos de dados.

In [13]:
df.dtypes

0
DATA     datetime64[ns]
UF       string[python]
VALOR             int64
dtype: object

Visualização dos dados filtrados e limpos.

In [14]:
df.head()

,DATA,UF,VALOR
1,1997-01-01,Rondônia,54392
2,1997-02-01,Rondônia,54822
3,1997-03-01,Rondônia,68027
4,1997-04-01,Rondônia,84671
5,1998-01-01,Rondônia,67600


### Conclusão: 
##### Após o processo de extração, manipulação e limpeza, os dados estão prontos para serem utilizados para a aplicação do dashboard.